In [1]:
import os
from jsonschema_rs import validator_for
import json

from jsonschema import validate


In [2]:
from pbcore.models import PBCore
SCHEMA='schemas/pbcore.schema.json'

with open(SCHEMA, 'w') as f:
    f.write(json.dumps(PBCore.model_json_schema(), indent=2))

In [3]:
DIRECTORY = 'pbcore-json'

with open(SCHEMA, 'r') as schema_file:
    schema = json.load(schema_file)

pb = validator_for(schema)

# with open(os.path.join(DIRECTORY, 'cpb-aacip_15-29p2p5g8.json'), 'r') as f:
# with open(os.path.join(DIRECTORY, 'cpb-aacip-15-nc5s756s32.json'), 'r') as f:
with open(os.path.join(DIRECTORY, 'cpb-aacip-d2a727119c1.json'), 'r') as f:
    json_data = json.load(f)

# json_data

# validate_json(json_data)
# validate(json_data, schema)
# pb.validate(json_data)
PBCore(**json_data)

PBCore(pbcoreDescriptionDocument=PBCoreDescriptionDocument(xsi_schemaLocation='http://www.pbcore.org/PBCore/PBCoreNamespace.html http://www.pbcore.org/xsd/pbcore-2.1.xsd', pbcoreAssetType=None, pbcoreAssetDate=None, pbcoreIdentifier=[PBCoreIdentifier(source='Sony Ci', ref=None, version=None, annotation=None, text='7a024c75053c4afa8bb15ce15d122b0b'), PBCoreIdentifier(source='http://americanarchiveinventory.org', ref=None, version=None, annotation=None, text='cpb-aacip-d2a727119c1'), PBCoreIdentifier(source='Local Identifier', ref=None, version=None, annotation=None, text='KUT-003521')], pbcoreTitle=[PBCoreTitle(startTime=None, endTime=None, timeAnnotation=None, source=None, ref=None, version=None, annotation=None, text='Border Radio', titleType='Program', titleTypeSource=None, titleTypeRef=None, titleTypeVersion=None, titleTypeAnnotation=None)], pbcoreSubject=None, pbcoreDescription=[PBCoreDescription(startTime=None, endTime=None, timeAnnotation=None, source=None, ref=None, version=None

In [4]:
errors = []
success = []
for path, subdirs, files in os.walk(DIRECTORY):
    for name in files:
        if name.startswith('cpb-aacip') and name.endswith('.json'):
            with open(os.path.join(path, name), 'r') as f:
                json_data = json.load(f)
                try:
                    # validate(json_data, schema) # jsonschema
                    pb.validate(json_data) # jsonschema_rs
                    # PBCore.model_validate(json_data) # pydantic
                    # print(f"{name} is valid.")
                    success.append(name)
                except Exception as e:
                    print(f"{name} is invalid: {e.message}")
                    errors.append((name, e.message))


In [5]:
len(success), len(errors), f'{(len(success)/(len(success) + len(errors))):.2%}'

(4088, 0, '100.00%')

In [6]:
mvp={
        "pbcoreDescriptionDocument": {
            "xsi:schemaLocation": "http://www.example.com/schema",
            "pbcoreIdentifier": [{"text": "123", "source": "example"}],
            "pbcoreTitle": [{"text": "Title"}],
            "pbcoreDescription": [{"text": ""}],
        },
    }
# del mvp['pbcoreDescriptionDocument']['pbcoreIdentifier'][0]['source']

# pb.validate(mvp)

In [7]:
PBCore.model_validate(mvp)

PBCore(pbcoreDescriptionDocument=PBCoreDescriptionDocument(xsi_schemaLocation='http://www.example.com/schema', pbcoreAssetType=None, pbcoreAssetDate=None, pbcoreIdentifier=[PBCoreIdentifier(source='example', ref=None, version=None, annotation=None, text='123')], pbcoreTitle=[PBCoreTitle(startTime=None, endTime=None, timeAnnotation=None, source=None, ref=None, version=None, annotation=None, text='Title', titleType=None, titleTypeSource=None, titleTypeRef=None, titleTypeVersion=None, titleTypeAnnotation=None)], pbcoreSubject=None, pbcoreDescription=[PBCoreDescription(startTime=None, endTime=None, timeAnnotation=None, source=None, ref=None, version=None, annotation=None, text='', descriptionType=None, descriptionTypeSource=None, descriptionTypeRef=None, descriptionTypeVersion=None, descriptionTypeAnnotation=None, segmentType=None, segmentTypeSource=None, segmentTypeRef=None, segmentTypeVersion=None, segmentTypeAnnotation=None)], pbcoreGenre=None, pbcoreRelation=None, pbcoreCoverage=None, 

In [8]:
p = PBCore(**mvp)


In [9]:
p.pbcoreDescriptionDocument.pbcoreIdentifier


[PBCoreIdentifier(source='example', ref=None, version=None, annotation=None, text='123')]